In [109]:
import numpy as np

test_inputs = \
    """0,9 -> 5,9
8,0 -> 0,8
9,4 -> 3,4
2,2 -> 2,1
7,0 -> 7,4
6,4 -> 2,0
0,9 -> 2,9
3,4 -> 1,4
0,0 -> 8,8
5,5 -> 8,2"""

with open("./inputs/d5p1.txt", "r") as f:
    raw_inputs = f.read()

input = raw_inputs
use_test = False
if use_test:
    input = test_inputs

input_lines = input.splitlines()

input = [coord.split("->") for coord in input_lines]
input = [[x[0].split(","), x[1].split(",")] for x in input]

input = np.array(input).astype(int)
max_dim = input.max()
hit_map = np.zeros((max_dim + 1, max_dim +1), dtype=int)

print(input.shape)
print(hit_map.shape)
print(input)


(500, 2, 2)
(991, 991)
[[[955 125]
  [151 929]]

 [[830 251]
  [526 555]]

 [[182 185]
  [ 13  16]]

 ...

 [[667  22]
  [308  22]]

 [[831 782]
  [245 196]]

 [[759 813]
  [225 279]]]


In [110]:
# Filter out only Vertical and Horizontal lines
# List of first coordinate:   input[:,0,:])
# List of second coordinates: input[:,1,:])
horiz_inputs = (input[:,0,0] == input[:,1,0])
vert_inputs = (input[:,0,1] == input[:,1,1])
ortho_inputs = input[vert_inputs | horiz_inputs]

# Fill in points along line
def fill_points(p1, p2):
    x_offset = p2[0] - p1[0]
    y_offset = p2[1] - p1[1]

    # Same point case
    if x_offset == 0 and y_offset == 0:
        # Just return p1 as it is the same as p2
        return np.array(p1)

    # Different X case
    if x_offset != 0 and y_offset == 0:
        line_points = np.empty((abs(x_offset)-1, 2), dtype=int)
        sign = 1 if x_offset > 0 else -1
        for x in range(1, abs(x_offset)):
            line_points[x-1] = np.array([p1[0] + x*sign, p1[1]])
        return line_points

    # Different Y case
    if x_offset == 0 and y_offset != 0:
        line_points = np.empty((abs(y_offset)-1, 2), dtype=int)
        sign = 1 if y_offset > 0 else -1
        for y in range(1, abs(y_offset)):
            line_points[y-1] = np.array([p1[0], p1[1] + y*sign])
        return line_points

    # Different X and Y case 
    if x_offset != 0 and y_offset != 0:
        assert abs(x_offset) == abs(y_offset)

# Apply lines to hit_map (+1 per hit)
inner_points = []
for input in input:
    inner_points.append(fill_points(input[0], input[1]))

for points in inner_points:
    for point in points:
        hit_map[point[0], point[1]] += 1

for points in ortho_inputs:
    for point in points:
        hit_map[point[0], point[1]] += 1

hit_map_ge2 = np.where(hit_map >= 2, 1, 0)
print(np.sum(hit_map_ge2))

# Count number of places that there are two or more hits

TypeError: 'NoneType' object is not iterable